<a href="https://colab.research.google.com/github/nbadiuc/Automatic-Text-Simplification-Based-On-NER/blob/main/Thesis_Automatic_Text_Simplification_Based_On_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The baseline model

In [10]:
!pip install torch
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 41.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=c809c774262a94057ab25a342e7a2f91801a5aae5170f858fa72d8b70b9980ec
  Sto

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
baseline_tokenizer = AutoTokenizer.from_pretrained("orzhan/rugpt3-simplify-large", padding_side='left')
baseline_model = AutoModelForCausalLM.from_pretrained("orzhan/rugpt3-simplify-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
from transformers import pipeline
baseline_simplifier = pipeline("text-generation", model=baseline_model, tokenizer=baseline_tokenizer)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [13]:
text = "<s>Родившийся 12 июня 1915 года американский банкир, государственный деятель, глобалист и глава дома Рокфеллеров Дэвид Рокфеллер старший () — внук нефтяного магната и первого в истории долларового миллиардера Джона Дэвисона Рокфеллера, основателя Стандарт Ойл, младший брат 41-го вице-президента США Нельсона Рокфеллера и 37-го губернатора Арканзаса Уинтропа О. Рокфеллера.<Simplify:>"
while text[-7:] != 'EndText':
    text = baseline_simplifier(text)[0]['generated_text']
print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 97, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 98, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 99, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasi

<s>Родившийся 12 июня 1915 года американский банкир, государственный деятель, глобалист и глава дома Рокфеллеров Дэвид Рокфеллер старший () — внук нефтяного магната и первого в истории долларового миллиардера Джона Дэвисона Рокфеллера, основателя Стандарт Ойл, младший брат 41-го вице-президента США Нельсона Рокфеллера и 37-го губернатора Арканзаса Уинтропа О. Рокфеллера.<Simplify:> Дэвид Рокфеллер старший (родился 12 июня 1915 года) - внук Джона Дэвиса Рокфеллера, основателя Standard Oil, и его жены, дочери Джона Рокфеллера. EndText


#BERT and synonym dictionary models

In [14]:
# download the dataset into the /tmp folder

import zipfile
import os

!wget "https://github.com/nerel-ds/NEREL/archive/refs/heads/master.zip" -O "/tmp/nerel.zip"

zip_ref = zipfile.ZipFile('/tmp/nerel.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2023-05-28 12:33:08--  https://github.com/nerel-ds/NEREL/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/nerel-ds/NEREL/zip/refs/heads/master [following]
--2023-05-28 12:33:08--  https://codeload.github.com/nerel-ds/NEREL/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/nerel.zip’

/tmp/nerel.zip          [     <=>            ]   6.14M  6.96MB/s    in 0.9s    

2023-05-28 12:33:09 (6.96 MB/s) - ‘/tmp/nerel.zip’ saved [6443546]



In [15]:
# reading a text and its annotation

def read_text(text_location="/tmp/NEREL-master/NEREL-v1.1/train/023.txt"):

    with open(text_location) as f:
        lines = f.readlines()
    
    # getting rid of the titles of the articles - they are not full sentences
    offset = len(lines[0]) # title length

    text = ''
    for i in range(1, len(lines)):
        text += lines[i]

    ann_location = text_location[:-3] + "ann"
    with open(ann_location) as f:
        ann = f.readlines()

    return text, ann, offset

In [16]:
text, ann, offset = read_text()
print("Text:")
print(text)
print("Annotation:")
print(ann)
print("Offset:")
print(offset)

Text:
Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal. На работу в Facebook он выйдет в сентябре. Ранее Эберсман был финансовым директором биотехнологической компании Genentech.

Эберсман подчеркнул, что видит много общего между Facebook и Genentech. В частности, это две быстрорастущие компании с сильной корпоративной культурой. Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.

В компании Genentech Дэвид Эберсман проработал 15 лет. Ее финансовым директором он стал в 2006 году. На этой должности Эберсман проработал до апреля 2009 года, когда Roche Holding купил Genentech.

По данным The Wall Street Journal, на должность финансового директора Facebook претендовали 11 кандидатов. Каждый из них прошел собеседование с основателем Facebook Марком Цукербергом (Mark Zuckerberg) и другими руководителями компании.

Ранее финансовым директором Facebook был Гидеон Ю (Gideon Yu). О

In [17]:
# using annotation, generates a mask that contains 1s in the places corresponding to named entities and 0s otherwise
def generate_NE_mask(text, ann, offset):
    mask = [0] * len(text)
    for line in ann:
        if line[0] == 'T': # "T" - terms; in annotation, they correspond to named entites
            start_ind, end_ind = line.split('\t')[1].split(' ')[1:]
            #named_entity = line.split('\t')[2].strip()
            start_ind = int(start_ind) - offset
            end_ind = int(end_ind) - offset
            if start_ind >= 0:
                for i in range(start_ind, end_ind):
                    mask[i] = 1
    return mask

In [18]:
mask = generate_NE_mask(text, ann, offset)
for i in range(min(len(mask), 250)):
    print(mask[i], end='')
print()
print(text[:min(len(mask), 250)])

1111111111111111111110000000000000000011111111011111111011111111101111111111111100111111111111110000000000001111111111111111111111100000000000000111111110000000000011111111110000000011111111000001111111111111111111110000000000000000000000000000011111
Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal. На работу в Facebook он выйдет в сентябре. Ранее Эберсман был финансовым директором биотехнологической компании Genen


In [19]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize

# splits text into sentences remembering start positions of each one
# (relative to text that does not contain name of the article, so, to match the positions in annotation, you need to add offset)
def split_text_into_sentences(text):
    sentences = sent_tokenize(text, language="russian")
    sentence_start_positions = []
    for i in range(len(sentences)):
        sentence_start_positions.append(text.find(sentences[i]))
    return sentences, sentence_start_positions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
sentences, sentence_start_positions = split_text_into_sentences(text)
for i in range(len(sentences)):
    print(sentence_start_positions[i], sentences[i])

0 Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal.
133 На работу в Facebook он выйдет в сентябре.
176 Ранее Эберсман был финансовым директором биотехнологической компании Genentech.
257 Эберсман подчеркнул, что видит много общего между Facebook и Genentech.
329 В частности, это две быстрорастущие компании с сильной корпоративной культурой.
409 Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.
494 В компании Genentech Дэвид Эберсман проработал 15 лет.
549 Ее финансовым директором он стал в 2006 году.
595 На этой должности Эберсман проработал до апреля 2009 года, когда Roche Holding купил Genentech.
692 По данным The Wall Street Journal, на должность финансового директора Facebook претендовали 11 кандидатов.
799 Каждый из них прошел собеседование с основателем Facebook Марком Цукербергом (Mark Zuckerberg) и другими руководителями компании.
930 Ранее финансовым директором 

In [21]:
from nltk.corpus import stopwords
import string

# splits sentences into words remembering start position of each one (relative to text without the name of article)
# gets rid of punctuation and stop words while doing so
def split_sentences_into_words(text, sentences, sentence_start_positions):

    words = []
    words_start_positions = []

    for i in range(len(sentences)):
        current_words = word_tokenize(sentences[i], language="russian")
        stop = set(stopwords.words('russian') + list(string.punctuation))
        current_words = [word for word in current_words if word not in stop]

        current_piece_of_text = text[sentence_start_positions[i]:]
        current_words_start_positions = []
        for word in current_words:
            current_words_start_positions.append(current_piece_of_text.find(word) + sentence_start_positions[i])
        words.append(current_words)
        words_start_positions.append(current_words_start_positions)

    return words, words_start_positions

In [22]:
words, words_start_positions = split_sentences_into_words(text, sentences, sentence_start_positions)
print(words)
print(words_start_positions)

[['Финансовым', 'директором', 'социальной', 'сети', 'Facebook', 'назначен', '39-летний', 'Дэвид', 'Эберсман', 'David', 'Ebersman', 'сообщает', 'The', 'Wall', 'Street', 'Journal'], ['На', 'работу', 'Facebook', 'выйдет', 'сентябре'], ['Ранее', 'Эберсман', 'финансовым', 'директором', 'биотехнологической', 'компании', 'Genentech'], ['Эберсман', 'подчеркнул', 'видит', 'общего', 'Facebook', 'Genentech'], ['В', 'частности', 'это', 'две', 'быстрорастущие', 'компании', 'сильной', 'корпоративной', 'культурой'], ['Также', 'заявил', 'Facebook', 'ожидает', '70-процентное', 'увеличение', 'выручки', '2009', 'году'], ['В', 'компании', 'Genentech', 'Дэвид', 'Эберсман', 'проработал', '15', 'лет'], ['Ее', 'финансовым', 'директором', 'стал', '2006', 'году'], ['На', 'должности', 'Эберсман', 'проработал', 'апреля', '2009', 'года', 'Roche', 'Holding', 'купил', 'Genentech'], ['По', 'данным', 'The', 'Wall', 'Street', 'Journal', 'должность', 'финансового', 'директора', 'Facebook', 'претендовали', '11', 'кандида

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import pandas as pd
word_frequencies = pd.read_csv('/content/drive/My Drive/freqrnc2011.csv', sep='\t', index_col='Lemma')

In [49]:
def word_complexity(word):
    if word in word_frequencies.index:
        word_frequency = word_frequencies.loc[word]['Freq(ipm)']
        if isinstance(word_frequency, pd.core.series.Series):
            word_frequency = max(word_frequency)
        return 15/word_frequency + len(word)/14
    else:
        return len(word)/7

In [26]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from transformers import pipeline
bert_simplifier = pipeline("fill-mask", model="bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def bert_model(initial_sentence, simplified_sentence, masked_word, word_start_position_in_initial_sentence, word_start_position_in_simplified_sentence):
    masked_sentence = initial_sentence[:word_start_position_in_initial_sentence] + "[MASK]" + initial_sentence[word_start_position_in_initial_sentence + len(masked_word):]
    #print(initial_sentence)
    #print(masked_sentence)
    results = bert_simplifier(masked_sentence)
    #print(results)
    for result in results:
        suggested_word = result['token_str']
        normalized_suggested_word = morph.parse(suggested_word)[0].normal_form
        if word_complexity(normalized_suggested_word) < word_complexity(masked_word):
            #print(word_complexity(normalized_suggested_word), word_complexity(masked_word))
            simplified_sentence = simplified_sentence[:word_start_position_in_simplified_sentence] + suggested_word + simplified_sentence[word_start_position_in_simplified_sentence + len(masked_word):]
            return simplified_sentence, len(suggested_word) - len(masked_word)
    return simplified_sentence, 0

In [ ]:
import pandas

def simplify(text_location="/tmp/NEREL-master/NEREL-v1.1/train/023.txt", method="BERT"):

    text, ann, offset = read_text(text_location)
    mask = generate_NE_mask(text, ann, offset)
    sentences, sentence_start_positions = split_text_into_sentences(text)
    words, words_start_positions = split_sentences_into_words(text, sentences, sentence_start_positions)

    simplified_sentences = []

    if method == "BERT":
        for sentence_ind in range(len(words)):
            simplified_sentence = sentences[sentence_ind]
            words_start_positions_in_simplified_sentence = words_start_positions[sentence_ind][:]
            for word_ind in range(len(words[sentence_ind])):
                words_start_positions_in_simplified_sentence[word_ind] -= sentence_start_positions[sentence_ind]
            for word_ind in range(len(words[sentence_ind])):
                current_word = words[sentence_ind][word_ind]
                if mask[words_start_positions[sentence_ind][word_ind]] == 0: # if the current word is not part of a named entity
                    normalized_current_word = morph.parse(current_word)[0].normal_form # to normal form
                    if word_complexity(normalized_current_word) > 1:
                        current_word_start_position_in_initial_sentence = words_start_positions[sentence_ind][word_ind] - sentence_start_positions[sentence_ind]
                        current_word_start_position_in_simplified_sentence = words_start_positions_in_simplified_sentence[word_ind]
                        simplified_sentence, simplification_offset = bert_model(sentences[sentence_ind], simplified_sentence, current_word, current_word_start_position_in_initial_sentence, current_word_start_position_in_simplified_sentence)
                        #print(simplified_sentence)
                        #print()
                        for new_word_ind in range(word_ind + 1, len(words[sentence_ind])):
                            words_start_positions_in_simplified_sentence[new_word_ind] += simplification_offset

In [ ]:
simplify()

In [32]:
import requests
import json

def get_synonyms(word):
    url = 'https://dictionary.yandex.net/api/v1/dicservice.json/lookup'
    params = {
        'key': 'dict.1.1.20230528T133010Z.158a1e5336183ca0.4e03e68250b1dfaa01b77acbf3439b0e45d1a614',
        'lang': 'ru-ru',
        'text': word
    }
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    try:
        synonyms_list = []
        for i in data['def'][0]['tr']:
            for j in i['syn']:
                synonyms_list.append(j['text'])
            synonyms_list.append(i['text'])
        return synonyms_list
    except:
        return []

In [62]:
def get_declension(word):
    p = morph.parse(word)[0]
    tags = {}
    if p.tag.POS == "VERB":
        tags = {p.tag.gender, p.tag.number, p.tag.tense}
        

    return p.tag.case, p.tag.gender, p.tag.number, p.tag.tense

get_declension("красноречивый")

OpencorporaTag('ADJF,Qual masc,sing,nomn')

In [81]:
word1 = "говорил"
p1 = morph.parse(word1)[0]
tags = {p1.tag.gender, p1.tag.number, p1.tag.tense}
word2 = "пробовать"
p2 = morph.parse(word2)[0]
print(p2.inflect(tags))

masc
Parse(word='пробовал', tag=OpencorporaTag('VERB,impf,tran masc,sing,past,indc'), normal_form='пробовать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'пробовал', 171, 7),))


In [38]:
def make_declension(word, case="nomn"):
    p = morph.parse(word)[0]
    return p.inflect({case}).word

In [45]:
def synonym_simplifier(word):
  declension = get_declension(word)
  nomn = make_declension(word)
  result = nomn
  synonyms = get_synonyms(nomn)
  print(synonyms)
  for synonym in synonyms:
    if word_complexity(synonym) < word_complexity(result):
      result = synonym
  return make_declension(result, declension)


In [57]:
synonym_simplifier("красноречивого")

['выразительный', 'экспрессивный', 'многозначительный', 'многоговорящий', 'убедительный', 'велеречивый', 'речистый', 'колоритный', 'сочный', 'красочный', 'образный', 'живой', 'живописный', 'яркий', 'словесный', 'ораторский', 'значительный', 'знаменательный', 'рельефный', 'выпуклый']


'живой'